In [106]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

import pickle as pickle

import sys
path = '../../scripts/'
sys.path.insert(0,path)
from ripser_to_dict import *
#from plot_persistence import *

In [107]:
with open('params.pickle', 'rb') as f:
    params = pickle.load(f)

## Functions

In [108]:
def get_label(string,params):
    shapes = params['shapes']
    label = 0
    for name in shapes['names']:
        if name == string:
            break
        else:
            label += 1
    return label

In [114]:
def create_circle_lower_distance_matrix(seed,params):
    
    # extract parameters
    shapes = params['shapes']
    label = get_label('circle', params)
    dev = shapes['deviations'][label]
    n_points = shapes['n_points']
    
    # seed the generator
    np.random.seed(seed=seed)
    
    # construct unit circle, r=1±dev, 0<theta<2*pi
    r = 1. - dev / 2. + dev * np.random.rand(n_points)
    theta = 2. * np.pi * np.random.rand(n_points)
    
    # transform into Euclidean
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    
    # calculate distances
    lower_distance_matrix = np.zeros((n_points,n_points))
    for i in range(n_points):
        for j in range(n_points - i):
            lower_distance_matrix[i,j] = np.sqrt((x[j] - x[i])**2 + (y[j] - y[i])**2)
    lower_distance_matrix = np.flipud(lower_distance_matrix)        
            
    return lower_distance_matrix

In [110]:
def create_sphere_upper_distance_matrix(seed,params):
    
    # extract parameters
    shapes = params['shapes']
    label = get_label('sphere', params)
    dev = shapes['deviations'][label]
    n_points = shapes['n_points']
    
    # seed the generator
    np.random.seed(seed=seed)
    
    # construct unit sphere in spherical coordinates: r=1±dev, 0<theta<pi, 0<phi<2*pi
    r = 1. - dev / 2. + dev * np.random.rand(n_points)
    theta = np.pi * np.random.rand(n_points)
    phi = 2. * np.pi * np.random.rand(n_points)
    
    # transform into Euclidean
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    
    # calculate distances
    upper_distance_matrix = np.zeros((n_points,n_points))
    for i in range(n_points):
        for j in range(n_points - i):
            upper_distance_matrix[i,j] = np.sqrt((x[j] - x[i])**2 + (y[j] - y[i])**2 + (z[j] - [z[i]])**2)
            
    return upper_distance_matrix

In [111]:
def create_torus_upper_distance_matrix(seed,params):
    
    # extract parameters
    shapes = params['shapes']
    label = get_label('torus', params)
    dev = shapes['deviations'][label]
    n_points = shapes['n_points']
    
    # seed the generator
    np.random.seed(seed=seed)
    
    # create noise
    A = 1. - dev / 2. +  dev * np.random.rand(n_points)
    
    # construct 0<theta<2*pi, 0<phi<2*pi
    theta = 2. * np.pi * np.random.rand(n_points)
    phi = 2. * np.pi * np.random.rand(n_points)
    
    # construct torus in R4
    x = A * np.cos(theta)
    y = A * np.sin(theta)
    z = A * np.cos(phi)
    t = A * np.sin(phi)
    
    # calculate distances
    upper_distance_matrix = np.zeros((n_points,n_points))
    for i in range(n_points):
        for j in range(n_points - i):
            upper_distance_matrix[i,j] = np.sqrt((x[j] - x[i])**2 + (y[j] - y[i])**2\
                                                 + (z[j] - [z[i]])**2 + (t[j] - t[i])**2)
            
    return upper_distance_matrix

## Run

In [115]:
for shape in params['shapes']['names']:
    for seed in params['shapes']['all_seeds']:
        if shape == 'circle':
            distance_matrix = create_circle_lower_distance_matrix(seed,params)
        elif shape == 'sphere':
            distance_matrix = create_sphere_upper_distance_matrix(seed,params)
        elif shape == 'torus':
            distance_matrix = create_torus_upper_distance_matrix(seed,params)
        np.savetxt(params['paths']['code_to_data'] + 'temp/distance.dat', distance_matrix, delimiter=',')
        
        print(distance_matrix)
        
        !cd {params['paths']['code_to_ripser']}; \
        ./ripser --help; \
        ./ripser --format upper-distance --dim {params['ripser']['dim']} --threshold {params['ripser']['threshold']} \
        {params['paths']['ripser_to_data']}temp/distance.dat \
        > {params['paths']['ripser_to_data']}temp/persistence.txt

        persistence = ripser_to_dict(params['paths']['code_to_data'] + 'temp/persistence.txt', params).copy()

        with open(params['paths']['code_to_data'] + 'persistences/' + shape + '/' + str(seed) + '.pickle', 'wb') as f:
            pickle.dump(persistence, f)

[[ 0.77385642  0.          0.         ...,  0.          0.          0.        ]
 [ 1.56993149  1.66982954  0.         ...,  0.          0.          0.        ]
 [ 0.28000277  1.74898667  0.13571713 ...,  0.          0.          0.        ]
 ..., 
 [ 0.41460638  1.7106786   0.         ...,  0.13571713  0.          0.        ]
 [ 1.89457333  0.          1.7106786  ...,  1.74898667  1.66982954  0.        ]
 [ 0.          1.89457333  0.41460638 ...,  0.28000277  1.56993149
   0.77385642]]
Usage: ripser [options] [filename]

Options:

  --help           print this screen
  --format         use the specified file format for the input. Options are:
                     lower-distance (lower triangular distance matrix; default)
                     upper-distance (upper triangular distance matrix)
                     distance       (full distance matrix)
                     point-cloud    (point cloud in Euclidean space)
                     dipha          (distance matrix in DIPHA file form

RuntimeError: dictionary changed size during iteration